# Яндекс Музыка

**Цель исследования:** 
На основе данных Яндекс Музыки сравнить поведение пользователей Москвы и Санкт-Петербурга и проверить три гипотезы:
* Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
* В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
* Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Входные данные:** Данные о поведении пользователей из файла yandex_music_project.csv.

**Описание данных:**

`userID` — идентификатор пользователя; <br/>
`Track` — название трека; <br/>
`artist` — имя исполнителя; <br/>
`genre` — название жанра; <br/>
`City` — город пользователя; <br/>
`time` — время начала прослушивания; <br/>
`Day` — день недели.

<a id="enter"></a>
**Содержание:**

1. [Обзор данных](#start)
2. [Предобработка данных](#preprocessing)
    * [2.1 Стиль заголовков](#style)
    * [2.2 Пропуски значений](#gap)
    * [2.3 Дубликаты](#duplicates)
    * [2.4 Промежуточные выводы](#concl1)
3. [Проверка гипотез](#research)
    * [3.1 Сравнение поведения пользователей двух столиц](#comp)
    * [3.2  Музыка в начале и в конце недели](#wday)
    * [3.3 Жанровые предпочтения в Москве и Петербурге](#gpref)  
4. [Общий вывод](#conclusion)

In [1]:
# импорт библиотеки pandas
import pandas as pd

<a id="start"></a>

## Обзор данных

Прочтем файл `yandex_music_project.csv` из папки `/datasets` и сохраните его в переменной `df`:

In [2]:
df=pd.read_csv('/datasets/yandex_music_project.csv')

Выведем первые 5 строк и получим общую информацию о таблице.

In [3]:
display(df.head(5))
df.info()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Промежуточные выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

[К содержанию](#enter)

<a id="preprocessing"></a>

## Предобработка данных

Исправим стили в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

<a id="style"></a>

### Стиль заголовков

Выведем на экран названия столбцов:

In [4]:
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Приведем названия в соответствие с хорошим стилем:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

In [5]:
df=df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

<a id="gap"></a>

### Пропуски значений

Посчитаем, сколько в таблице пропущенных значений.

In [6]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого созддим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [7]:
columns_to_replace=['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column]=df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [8]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[К содержанию](#enter)

<a id="duplicates"></a>

### Дубликаты

Посчитайте явные дубликаты в таблице:

In [9]:
df.duplicated().sum()

3826

Удалим явные дубликаты:

In [10]:
df=df.drop_duplicates()

Ещё раз посчитаем явные дубликаты в таблице чтобы убедиться в их отсутствии:

In [11]:
df.duplicated().sum()

0

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем уникальные названия жанров из столбца `genre` отсортированному по алфавиту.

In [12]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Как можно видеть из списка, присутствуют следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Исправим колонку `genre` в таблице `df` так чтобы вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [13]:
duplicates=['hip', 'hop', 'hip-hop']
genre='hiphop'
df['genre']=df['genre'].replace(duplicates, genre)

Выведем вновь отсортированный список жанров:

In [14]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

<a id="concl1"></a>

### Промежуточные выводы

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Заголовки исправлены, дубликаты устранены.

Пропущенные значения вы заменены `'unknown'`.

[К содержанию](#enter)

<a id="research"></a>

## Проверка гипотез

<a id="comp"></a>

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для начала оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [15]:
df.groupby('city')['city'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и посчитаем прослушивания в понедельник, среду и пятницу. Учтем, что в данных есть информация о прослушиваниях только за эти дни.

In [16]:
df.groupby('day')['day'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. 

In [17]:
def number_tracks(day,city):
    track_list=df[df['day']==day]
    track_list=track_list[track_list['city']==city] 
    track_list_count=track_list['user_id'].count()
    return track_list_count

Вызовем фукцию 6 раз для того чтобы узнать число прослушиваний для каждого города и одного из трех дней.

In [18]:
number_tracks('Monday', 'Moscow')

15740

In [19]:
number_tracks('Monday', 'Saint-Petersburg')

5614

In [20]:
number_tracks('Wednesday', 'Moscow')

11056

In [21]:
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [22]:
number_tracks('Friday', 'Moscow')

15945

In [23]:
number_tracks('Friday', 'Saint-Petersburg')

5895

Создадим с помощью конструктора `pd.DataFrame` таблицу, где 
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [24]:
columns=['city', 'monday', 'wednesday', 'friday']
data=[['Mocsow', 15740, 11050, 15945],['Saint-Petersburg', 5614, 7003, 5895]]
number_tracks=pd.DataFrame(data=data, columns=columns)
print(number_tracks)

               city  monday  wednesday  friday
0            Mocsow   15740      11050   15945
1  Saint-Petersburg    5614       7003    5895


**Промежуточные выводы:**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

[К содержанию](#enter)

<a id="wday"></a>

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [25]:
moscow_general=df[df['city']=='Moscow']
spb_general=df[df['city']=='Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [26]:
def genre_weekday(df, day, time1, time2):# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
    genre_df=df[df['day']==day]
    genre_df=genre_df[genre_df['time']>time1]
    genre_df=genre_df[genre_df['time']<time2] 
    genre_df_grouped=genre_df.groupby('genre')['genre'].count() # 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
    return genre_df_sorted[:10]

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [27]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [28]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [29]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [30]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Промежуточные выводы:**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

[К содержанию](#enter)

<a id="gpref"></a>

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [31]:
moscow_general_gruoped=moscow_general.groupby('genre')['genre'].count()
moscow_genres=moscow_general_gruoped.sort_values(ascending=False)
print(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


Теперь повторим то же и для Петербурга.

Сгруппируем таблицу `spb_general` по жанру. Посчитаем прослушивания треков каждого жанра. Результат отсортируем в порядке убывания и сохраните в таблице `spb_genres`:

In [32]:
spb_general_gruoped=spb_general.groupby('genre')['genre'].count()
spb_genres=spb_general_gruoped.sort_values(ascending=False)
print(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


**Промежуточные выводы:**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 

<a id="conclusion"></a>

## Общий вывод

Проверены три гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

[К содержанию](#enter)